数据集：《统计学习方法》数据

训练集数量：10

测试集数量：0

层数：3

运行结果：正确率：100%；运行时长：<1m

In [1]:
import numpy as np

In [2]:
def loadData(fileName):
    '''
    加载文件
    :param fileName:要加载的文件路径
    :return: 数据集和标签集
    '''
    #存放数据及标记
    dataArr = []; labelArr = []
    #读取文件
    fr = open(fileName)
    #遍历文件中的每一行
    for line in fr.readlines():
        #获取当前行，并按“，”切割成字段放入列表中
        #strip：去掉每行字符串首尾指定的字符（默认空格或换行符）
        #split：按照指定的字符将字符串切割成每个字段，返回列表形式
        curLine = line.strip().split(',')
        #将curLine[1]数据放入数据集中, 将原先字符串形式的数据转换为整型
        dataArr.append(int(curLine[1]))

        #将curLine[0]标记信息放入标记集中, 放入的同时将标记转换为整型
        #二分类任务, 标签1设置为1，反之为-1
        if int(curLine[0]) == 1:
            labelArr.append(1)
        else:
            labelArr.append(-1)
    #返回数据集和标记
    return dataArr, labelArr

In [3]:
def calc_e_Gx(trainDataArr, trainLabelArr, n, div, rule, D):
    '''
    计算分类错误率
    :param trainDataArr: 训练数据集数组
    :param trainLabelArr: 训练标签集数组
    :param n: 要操作的特征
    :param div: 划分点
    :param rule: 正反例标签
    :param D: 权值分布D
    :return: 预测结果，分类误差率
    '''
    #初始化分类误差率为0
    e = 0
    #训练数据数组
    x = trainDataArr
    #同样将标签也转换成数组格式，x和y的转换只是单纯为了提高运行速度
    #测试过相对直接操作而言性能提升很大
    y = trainLabelArr
    predict = []

    #依据小于和大于的标签依据实际情况会不同，在这里直接进行设置
    if rule == 'LisOne':    L = 1; H = -1
    else:                   L = -1; H = 1

    #遍历所有样本的特征m
    for i in range(trainDataArr.shape[0]):
        if x[i] < div:
            #如果小于划分点，则预测为L
            #如果设置小于div为1，那么L就是1，
            #如果设置小于div为-1，L就是-1
            predict.append(L)
            #如果预测错误，分类错误率要加上该分错的样本的权值（8.1式）
            if y[i] != L: e += D[i]
        elif x[i] >= div:
            #与上面思想一样
            predict.append(H)
            if y[i] != H: e += D[i]
    #返回预测结果和分类错误率e
    #预测结果其实是为了后面做准备的，在算法8.1第四步式8.4中exp内部有个Gx，要用在那个地方
    #以此来更新新的D
    return np.array(predict), e

In [4]:
def createSigleBoostingTree(trainDataArr, trainLabelArr, D):
    '''
    创建单层提升树
    :param trainDataArr: 训练数据集数组
    :param trainLabelArr: 训练标签集数组
    :param D: 算法8.1中的D
    :return: 创建的单层提升树
    '''

    #获得样本数目及特征数量, 本例特征数量为1
    n = 1
    #单层树的字典，用于存放当前层提升树的参数
    #也可以认为该字典代表了一层提升树
    sigleBoostTree = {}
    #初始化分类误差率，分类误差率在算法8.1步骤（2）（b）有提到
    #误差率最高也只能100%，因此初始化为1
    sigleBoostTree['e'] = 1

    #对每一个特征进行遍历，寻找用于划分的最合适的特征；因为本例只有一个特征，可以忽略此循环
    for i in range(n):
        #因为特征的取值范围在0-9，因此分切分时分为下面几种取值进行切割
        for div in [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]:
            #在单个特征内对正反例进行划分时，有两种情况：
            #可能是小于某值的为1，大于某值得为-1，也可能小于某值得是-1，反之为1
            #因此在寻找最佳提升树的同时对于两种情况也需要遍历运行
            #LisOne：Low is one：小于某值得是1
            #HisOne：High is one：大于某值得是1
            for rule in ['LisOne', 'HisOne']:
                #按照第i个特征，以值div进行切割，进行当前设置得到的预测和分类错误率
                #【4】估计基学习器ht的误差
                Gx, e = calc_e_Gx(trainDataArr, trainLabelArr, i, div, rule, D)
                #如果分类错误率e小于当前最小的e，那么将它作为最小的分类错误率保存（记录最小分类错误率的目的是为了找到符合要求的切割阈值）
                #【5】基学习器均满足e<0.5，代码中省略了这一判定
                if e < sigleBoostTree['e']:
                    sigleBoostTree['e'] = e
                    #同时也需要存储最优划分点、划分规则、预测结果、特征索引
                    #以便进行D更新和后续预测使用
                    sigleBoostTree['div'] = div
                    sigleBoostTree['rule'] = rule
                    sigleBoostTree['Gx'] = Gx
                    sigleBoostTree['feature'] = i
    #返回单层的提升树
    return sigleBoostTree

In [5]:
def createBosstingTree(trainDataList, trainLabelList, treeNum = 50):
    '''
    创建提升树
    创建算法依据“8.1.2 AdaBoost算法” 算法8.1
    :param trainDataList: 训练数据集
    :param trainLabelList: 训练测试集
    :param treeNum: 树的层数
    :return: 提升树
    '''
    #将数据和标签转化为数组形式
    trainDataArr = np.array(trainDataList)
    trainLabelArr = np.array(trainLabelList)
    #每增加一层树后，当前最终预测结果列表
    finallpredict = [0] * len(trainLabelArr)
    #获得训练集数量
    m = len(trainDataArr)

    #【1】初始化样本权值分布
    D = [1 / m] * m
    #初始化提升树列表，每个位置为一层
    tree = []
    #【2】循环进行T轮训练（循环创建提升树）
    #这里之所以叫“提升树”，是因为在本例中，AdaBoost算法使用的基分类器本质上是一个“决策树桩”
    for i in range(treeNum):
        #【3】基于分布Dt从数据集中训练出分类器，得到当前层的提升树
        curTree = createSigleBoostingTree(trainDataArr, trainLabelArr, D)
        #【6】确定基分类器的权重（当前层的alpha）
        alpha = 1/2 * np.log((1 - curTree['e']) / curTree['e'])
        #获得当前层的预测结果，用于下一步更新D
        Gx = curTree['Gx']

        #【7】更新样本分布，Zm是规范化因子，以确保D是一个分布
        #D是一个行向量，由wmi元素组成，然后D中元素求和为Zm，Zm规范化因子使得D是一个分布
        D = np.multiply(D, np.exp(-1 * alpha * np.multiply(trainLabelArr, Gx)))
        Zm = sum(D)
        D = D / Zm
        
        #在当前层参数中增加alpha参数，预测的时候需要用到
        curTree['alpha'] = alpha
        #将当前层添加到提升树索引中。
        tree.append(curTree)

        #-----以下代码用来辅助，可以去掉---------------
        #将结果加上当前层乘以α，得到截至目前循环的输出预测
        finallpredict += alpha * Gx
        #计算当前最终预测输出与实际标签之间的误差
        error = sum([1 for i in range(len(trainDataList)) if np.sign(finallpredict[i]) != trainLabelArr[i]])
        #计算当前误差率
        finallError = error / len(trainDataList)
        #如果误差为0，提前退出即可，因为没有必要再计算了
        if finallError == 0:
            print(finallpredict)
            print('iter:%d:%d, sigle error:%.4f, finall error:%.4f'%(i, treeNum, curTree['e'], finallError ))
            return tree
        #打印一些信息
        print(finallpredict)
        print('iter:%d:%d, sigle error:%.4f, finall error:%.4f'%(i, treeNum, curTree['e'], finallError ))
    #返回整个提升树
    return tree


In [6]:
def predict(x, div, rule, feature):
    '''
    输出单独层预测结果
    :param x: 预测样本
    :param div: 划分点
    :param rule: 划分规则
    :param feature: 进行操作的特征
    :return:
    '''
    #依据划分规则定义小于及大于划分点的标签
    if rule == 'LisOne':    L = 1; H = -1
    else:                   L = -1; H = 1

    #判断预测结果
    if x < div: return L
    else:   return H

In [7]:
def model_test(testDataList, testLabelList, tree):
    '''
    测试
    :param testDataList: 测试数据集
    :param testLabelList: 测试标签集
    :param tree: 提升树
    :return: 准确率
    '''
    #错误率计数值
    errorCnt = 0
    #遍历每一个测试样本
    for i in range(len(testDataList)):
        #预测结果值，初始为0
        result = 0
        #依据算法8.1式8.6
        #预测式子是一个求和式，对于每一层的结果都要进行一次累加
        #遍历每层的树
        for curTree in tree:
            #获取该层参数
            div = curTree['div']
            rule = curTree['rule']
            feature = curTree['feature']
            alpha = curTree['alpha']
            #将当前层结果加入预测中
            result += alpha * predict(testDataList[i], div, rule, feature)
        #预测结果取sign值，如果大于0 sign为1，反之为0
        if np.sign(result) != testLabelList[i]: errorCnt += 1
    #返回准确率
    return 1 - errorCnt / len(testDataList)

In [8]:
# 获取训练集
print('start read transSet')
trainDataList, trainLabelList = loadData('example.csv')

start read transSet


In [9]:
trainDataList

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
trainLabelList

[1, 1, 1, -1, -1, -1, 1, 1, 1, -1]

In [11]:
# 获取测试集
print('start read testSet')
testDataList, testLabelList = loadData('example.csv')

start read testSet


In [12]:
# 创建提升树
print('start init train')
tree = createBosstingTree(trainDataList, trainLabelList, 40)

start init train
[ 0.42364893  0.42364893  0.42364893 -0.42364893 -0.42364893 -0.42364893
 -0.42364893 -0.42364893 -0.42364893 -0.42364893]
iter:0:40, sigle error:0.3000, finall error:0.3000
[ 1.07329042  1.07329042  1.07329042  0.22599256  0.22599256  0.22599256
  0.22599256  0.22599256  0.22599256 -1.07329042]
iter:1:40, sigle error:0.2143, finall error:0.3000
[ 0.32125172  0.32125172  0.32125172 -0.52604614 -0.52604614 -0.52604614
  0.97803126  0.97803126  0.97803126 -0.32125172]
iter:2:40, sigle error:0.1818, finall error:0.0000


In [13]:
tree

[{'e': 0.30000000000000004,
  'div': 2.5,
  'rule': 'LisOne',
  'Gx': array([ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1]),
  'feature': 0,
  'alpha': 0.4236489301936017},
 {'e': 0.21428571428571427,
  'div': 8.5,
  'rule': 'LisOne',
  'Gx': array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1]),
  'feature': 0,
  'alpha': 0.6496414920651304},
 {'e': 0.18181818181818185,
  'div': 5.5,
  'rule': 'HisOne',
  'Gx': array([-1, -1, -1, -1, -1, -1,  1,  1,  1,  1]),
  'feature': 0,
  'alpha': 0.752038698388137}]

In [14]:
#测试
print('start to test')
accuracy = model_test(trainDataList, trainLabelList, tree)
print('the accuracy is:%d' % (accuracy * 100), '%')

start to test
the accuracy is:100 %
